In [ ]:
!sudo apt-get update
!apt-get install x11-utils > /dev/null 2>&1 
!pip install pyglet > /dev/null 2>&1 
!apt-get install -y xvfb python-opengl > /dev/null 2>&1

!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install colabgymrender==1.0.2
!wget http://www.atarimania.com/roms/Roms.rar
!mkdir /content/ROM/
!unrar e /content/Roms.rar /content/ROM/
!python -m atari_py.import_roms /content/ROM/

from IPython.display import clear_output
clear_output()

In [ ]:
# Importing the libraries

import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder    #records videos of episodes
import numpy as np
import matplotlib.pyplot as plt # Graphical library

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Configuring Pytorch

from collections import namedtuple, deque
from itertools import count
import math
import random
import copy

# WARNING: if you don't use this Notebook in Google Colab, comment out these two imports
from colabgymrender.recorder import Recorder # Allow to record videos in Google Colab
Recorder(gym.make("CartPole-v1"), './video') # Defining the video recorder
clear_output()

In [ ]:
# Test cell: check ai gym  environment + recording working as intended

env = gym.make("CartPole-v1")
file_path = 'video/video.mp4'
recorder = VideoRecorder(env, file_path)

observation = env.reset()
terminal = False
while not terminal:
  recorder.capture_frame()
  action = int(observation[2]>0)
  observation, reward, terminal, info = env.step(action)
  # Observation is position, velocity, angle, angular velocity

recorder.close()
env.close()

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayBuffer(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)
        self.old_memory = []

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
class DQN(nn.Module):

    def __init__(self, inputs, k, outputs, num_hidden, hidden_size):
        super(DQN, self).__init__()
        self.input_layer = nn.Linear(inputs * k, hidden_size)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden-1)])
        self.output_layer = nn.Linear(hidden_size, outputs)
    
    def forward(self, x):
        x.to(device)

        x = F.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = F.relu(layer(x))
        
        return self.output_layer(x)

    def set_weights(self, main_network):
        self.load_state_dict(main_network.state_dict())

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)

    # Can safely omit the condition below to check that not all states in the
    # sampled batch are terminal whenever the batch size is reasonable and
    # there is virtually no chance that all states in the sampled batch are 
    # terminal
    if sum(non_final_mask) > 0:
        non_final_next_states = torch.cat([s for s in batch.next_state
                                                    if s is not None])
    else:
        non_final_next_states = torch.empty(0,state_dim).to(device)

    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE , device=device)
    
    with torch.no_grad():
        # Once again can omit the condition if batch size is large enough
        if sum(non_final_mask) > 0:
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
        else:
            next_state_values = torch.zeros_like(next_state_values)

    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute loss
    loss = loss_fn(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()

    # Limit magnitude of gradient for update step
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)

    optimizer.step()

In [ ]:
NUM_EPISODES = 250
BATCH_SIZE = 128
GAMMA = 0.999

num_hidden_layers = 2
size_hidden_layers = 64

k=2

replay_mem_min_size =  1
replay_mem_max_size = 5000

model_update_interval = 1

target_network_update_interval = 100

start_epsilon = 1.0
epsilon = 1.0
min_epsilon = 0.01
epsilon_decay = 0

batch_size_increase = 2
max_batch_size = 400


# Get number of states and actions from gym action space
env = gym.make("CartPole-v1")
env.reset()
state_dim = len(env.state)    #x, x_dot, theta, theta_dot
n_actions = env.action_space.n
env.close()

policy_net = DQN(state_dim, k, n_actions, num_hidden_layers, size_hidden_layers).to(device)
target_net = copy.deepcopy(policy_net)
target_net.set_weights(policy_net)
target_net.eval()
                    
memory = ReplayBuffer(replay_mem_max_size)

optimizer = optim.Adam(policy_net.parameters(), amsgrad=True)
loss_fn = torch.nn.MSELoss()

def select_action(state, current_eps=0):

    sample = random.random()
    eps_threshold = current_eps

    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.

            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [ ]:
def plot_learning_curve(total_scores, label):
    means = np.mean(np.array(total_scores), axis=0)
    sds = np.std(np.array(total_scores), axis=0)
    plt.plot(np.arange(0,NUM_EPISODES, 1), means)
    plt.fill_between(np.arange(0,NUM_EPISODES,1), means+sds, means-sds, alpha=0.2)
    plt.ylabel("Mean Reward")
    plt.xlabel("Episode")
    plt.title(label)
    plt.legend()
    plt.show()

In [ ]:

def reset(i_decay_rate = 20, i_epsilon = 1.0, i_lr=0.01, i_target_network_update_interval=100, i_batch_size=8, i_mem_max_size=10000, i_num_hidden_layers=2, i_size_hidden_layers=64, i_k=4):
    global epsilon, epsilon_decay, policy_net, target_net, memory, optimizer, k, BATCH_SIZE
    epsilon = i_epsilon
    epsilon_decay = i_decay_rate
    k=i_k
    BATCH_SIZE = i_batch_size

    env.reset()
    env.close()

    policy_net = DQN(state_dim, i_k, n_actions, i_num_hidden_layers, size_hidden_layers).to(device)
    target_net = copy.deepcopy(policy_net)
    target_net.eval()
                        
    memory = ReplayBuffer(i_mem_max_size)
    optimizer = optim.Adam(policy_net.parameters())
    


total_scores = []

learning = True
reset()
total_steps = 0
scores = []

for i_episode in range(NUM_EPISODES):
  total_reward = 0

  # Initialize the environment and state
  env.reset()
  init_state = torch.tensor(env.state).float().unsqueeze(0).to(device)
  state_list = deque([init_state for _ in range(k)], maxlen=k)
  next_state = []

  for t in count():
      # Select and perform an action
      
      action = select_action(torch.cat(list(state_list), dim=1), epsilon)
      _, reward, done, _ = env.step(action.item())
      reward = torch.tensor([reward], device=device)

      total_reward += reward
      

      # Observe new state
      if not done:
          new_state = (torch.tensor(env.state).float().unsqueeze(0).to(device))
          next_state = torch.cat(list(state_list)[1:]+[new_state], dim=1)
      else:
          next_state = None

      
      # Store the transition in memory
      memory.push(torch.cat(list(state_list), dim=1), action, next_state, reward)

      # Perform one step of the optimization (on the policy network)
      if total_steps % model_update_interval == 0 and learning:
        optimize_model()
        
      if done:
          break
      
      # Move to the next state
      state_list.append(new_state)

      if total_steps % target_network_update_interval == 0:
          target_net.set_weights(policy_net)

      total_steps +=1
    

  if epsilon_decay > 0:
      epsilon = min_epsilon + (start_epsilon - min_epsilon) * math.exp(-1. * i_episode / epsilon_decay)
  
  BATCH_SIZE = min(BATCH_SIZE +batch_size_increase, max_batch_size)

  if i_episode % 10 == 0:
      print("episode ", i_episode, "/", NUM_EPISODES, " epsilon: ", epsilon, " lr: ", optimizer.param_groups[0]['lr'] ," batch size: ", BATCH_SIZE , " replay mem size: ", len(memory))
      if i_episode > 0 and len(memory) >= replay_mem_min_size + 1:
          print("                     score: ", total_reward)
      print("mean: ", np.mean(scores[-10:]))

  scores.append(total_reward)
  if (np.mean(scores[-10:]) >= 500):
      optimizer = optim.Adam(policy_net.parameters(), lr=0)
      learning = False

env.close()
total_scores.append(scores)
 
plot_learning_curve(total_scores, "DQN Learning Curve")


In [ ]:
## run an episode with trained agent and record video
## remember to change file_path name if you do not wish to overwrite an existing video

env = gym.make("CartPole-v1")
file_path = 'video/video.mp4'
recorder = VideoRecorder(env, file_path)

observation = env.reset()
done = False

state = torch.tensor(env.state).float().unsqueeze(0)
state_list = deque([init_state for _ in range(k)], maxlen=k)

duration = 0

while not done:
    recorder.capture_frame()

    # Select and perform an action
    action = select_action(torch.cat(list(state_list), dim=1))
    observation, reward, done, _ = env.step(action.item())
    duration += 1
    reward = torch.tensor([reward], device=device)

    # Observe new state
    state = torch.tensor(env.state).float().unsqueeze(0)
    state_list.append(state)

recorder.close()
env.close()
print("Episode duration: ", duration)